
<meta name="viewport" content="width=device-width, initial-scale=1">
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.2/css/all.min.css">
<link rel="stylesheet" href="../static/css/styles.css">


        
<!-- <body> -->
<!-- Navigation-->
<nav class="navbar navbar-expand-lg navbar-light fixed-top" id="mainNav">
    <div class="container px-4 px-lg-5">
        <a class="navbar-brand" href="../index.html">Home</a>
        <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarResponsive" aria-controls="navbarResponsive" aria-expanded="false" aria-label="Toggle navigation">
            Menu
            <i class="fas fa-bars"></i>
        </button>
        <div class="collapse navbar-collapse" id="navbarResponsive">
            <ul class="navbar-nav ms-auto py-4 py-lg-0">
                <li class="nav-item"><a class="nav-link px-lg-3 py-3 py-lg-4" href="../index.html">Executive Summary</a></li>
                <li class="nav-item"><a class="nav-link px-lg-3 py-3 py-lg-4" href="eda.html">Exploratory Data Analysis</a></li>
                <!-- <li class="nav-item"><a class="nav-link px-lg-3 py-3 py-lg-4" href="models.html">Model Construction & Validation</a></li> -->
                <li class="nav-item"><a class="nav-link px-lg-3 py-3 py-lg-4" href="initial_work.html">Reference: Model Development</a></li>
            </ul>
        </div>
    </div>
</nav>



<h2 id="title" style="text-align: center; width: 80%;">Model Construction and Validation</h2>

# Table of Contents

- [Description and Deliverables](#description-and-deliverables)
- [Exploratory Data Analysis Insights](#insights)
- [Modeling Strategies Recap](#modeling-strategies-recap)
  - [Cross-Validation Results](#cross-validation-results)
- [Model Building](#model-building)
  - [Logistic Regression](#logistic-regression)
  - [Decision Tree](#decision-tree)
  - [Random Forest](#random-forest)
  - [XGBoost](#xgboost)
- [Model Results](#model-results)
- [Appendix: Data Dictionary](#appendix-data-dictionary)
- [Appendix: Feature Engineering](#appendix-feature-engineering)
- [Appendix: Hyperparameter Tuning Grids](#appendix-hyperparameters)

<div class="button">
    <a href="eda.html">Back to Part 1<br>Exploratory Data Analysis</a>
</div>

<a id="description-and-deliverables"></a>

# Description and Deliverables
---

[Back to top](#)

This project builds on insights from exploratory data analysis (EDA) to **develop predictive models for employee attrition** at Salifort Motors. The primary objective was to identify employees at risk of leaving, enabling targeted retention strategies. The modeling process was designed to be rigorous, transparent, and accessible to both technical and non-technical stakeholders.

**Stakeholders:**  
The primary stakeholder is the Human Resources (HR) department, as they will use the results to inform retention strategies. Secondary stakeholders include C-suite executives who oversee company direction, managers implementing day-to-day retention efforts, employees (whose experiences and outcomes are directly affected), and, indirectly, customers—since employee satisfaction can impact customer satisfaction.

**Ethical Considerations:**  
- Ensure employee data privacy and confidentiality throughout the analysis.
- Avoid introducing or perpetuating bias in model predictions (e.g., not unfairly targeting specific groups).
- Maintain transparency in how predictions are generated and how they will be used in HR decision-making.

### This page summarizes the second part of the project: predictive model construction and validation, with interpretation of results.

In [1]:
# Import packages
import time
import joblib
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import plot_tree

from IPython.display import display, HTML

from sklearn.model_selection import (
    StratifiedKFold,
    cross_val_predict,
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    fbeta_score,
)

In [2]:
# Load dataset into a dataframe
df0 = pd.read_csv("../resources/HR_capstone_dataset.csv")

In [3]:
# Rename columns as needed
df0.rename(
    columns={
        "Department": "department",
        "Work_accident": "work_accident",
        "average_montly_hours": "average_monthly_hours",
        "time_spend_company": "tenure",
    },
    inplace=True,
)

In [4]:
# Drop duplicates and save resulting dataframe in a new variable as needed
df = df0.drop_duplicates()

In [5]:
# Determine the number of rows containing tenure outliers
q1 = df.tenure.quantile(0.25)
q3 = df.tenure.quantile(0.75)
iqr = q3 - q1
upper_bound = q3 + 1.5 * iqr

# # Filter the dataframe to find outliers
# outliers = df[df.tenure > upper_bound]

# # Display the number of outliers
# print(f"Number of tenure outliers: {len(outliers)}")
# print(f"Outliers percentage of total: {len(outliers) / len(df) * 100:.2f}%")

<a id="insights"></a>

# Exploratory Data Analysis Insights
---

[Back to Top](#)

#### A quick recap of the [exploratory data analysis](./eda.html):

The data suggests significant challenges with employee retention at this company. Two main groups of leavers emerge:

- **Underutilized and Dissatisfied:** Employees in this category worked on fewer projects and logged fewer hours than a typical full-time schedule, and reported lower satisfaction. These individuals may have been disengaged, assigned less work as they prepared to leave, or potentially subject to layoffs or terminations.
- **Overworked and Burned Out:** The second group managed a high number of projects (up to 7) and worked exceptionally long hours—sometimes nearing 80 hours per week. These employees exhibited very low satisfaction and rarely received promotions, suggesting that high demands without recognition or advancement led to burnout and resignation.

A majority of the workforce greatly exceeds the typical 40-hour work week (160–184 hours per month), pointing to a workplace culture that expects long hours. The combination of high workload and limited opportunities for advancement likely fuels dissatisfaction and increases the risk of turnover.

Performance evaluations show only a weak link to attrition; both those who left and those who stayed received similar review scores. This indicates that strong performance alone does not guarantee retention, especially if employees are overworked or lack opportunities for growth.

Other variables—such as department, salary, and work accidents—do not show strong predictive value for employee churn compared to satisfaction and workload. Overall, **the data points to issues with workload management** and limited career progression as the main factors driving employee turnover at this company.

<a id="modeling-strategies-recap"></a>

# Modeling Strategies Recap
---

[Back to top](#)

#### Approach

Given the business context, the modeling strategy **prioritized recall**—the ability to identify as many at-risk employees as possible—while also considering precision and model simplicity. Four model types were evaluated: Logistic Regression, Decision Tree, Random Forest, and XGBoost. Each model was developed using a systematic pipeline approach to ensure reproducibility and prevent data leakage.

Key steps included:
- **Data Preparation**: Outliers were removed for logistic regression, and all preprocessing was handled within scikit-learn `Pipelines`.
- **Cross-Validation**: Stratified K-Fold cross-validation was used throughout to ensure robust performance estimates, especially given class imbalance.
- **Hyperparameter Tuning**: `GridSearchCV` was used for simpler models, while `RandomizedSearchCV` was adopted for more complex models to balance thoroughness and computational efficiency. Details [here](#appendix-hyperparameters).
- **Feature Engineering**: Multiple rounds of feature engineering and selection were conducted, focusing on satisfaction, workload, tenure, and interaction terms identified as important in EDA. Details [here](#appendix-feature-engineering).
- **Model Evaluation**: In addition to standard metrics (recall, precision, F₁, ROC AUC), a custom F₂ score was introduced to explicitly weight recall four times higher than precision, reflecting the business priority of minimizing false negatives.

#### Lessons Learned

The modeling process was iterative and exploratory, reflecting both best practices and lessons learned:
- **Metric Selection Matters**: Initial experiments optimizing for AUC led to suboptimal recall, especially for logistic regression. Refocusing on recall as the primary metric improved model alignment with business goals.
- **Model Complexity vs. Interpretability**: Tree-based models (Random Forest, XGBoost) consistently outperformed logistic regression in overall metrics, but targeted feature engineering allowed logistic regression to become competitive while remaining highly interpretable.
- **Feature Engineering**: Additional features and interaction terms provided modest improvements, but the most significant gains came from careful feature selection and focusing on core predictors.
- **Efficiency and Automation**: Automating pipeline construction and model evaluation streamlined experimentation and reduced the risk of errors or data leakage.


#### Final Model Selection

With several high-performing models, explicit selection criteria were established:

1. Minimum recall threshold (≥ 0.935)
2. Minimum F₂ score (≥ 0.85)
3. Fewest features (for simplicity and interpretability)
4. Highest F₂ score
5. Highest precision

This approach ensured that the final models were not only accurate but also practical for deployment and stakeholder communication.

#### Next Steps

The selected models will be retrained on the full training set and evaluated on the holdout test set (`X_test`). Results—including confusion matrices, feature importances, and actionable insights—will be communicated to HR and leadership to inform retention strategies. The process and findings will be documented to support transparency, ethical use, and ongoing model improvement.

---

Following a focused exploratory data analysis, I entered model development with a clear sense of which features and relationships to explore. However, as this was only my second machine learning project, the learning curve was steep. I approached the modeling process ambitiously—iterating frequently, experimenting extensively, and making my share of mistakes along the way.

One of the key challenges was a lack of explicit early-stage success criteria. While I had always intended to compare four model types (Logistic Regression, Decision Tree, Random Forest, and XGBoost), I initially aimed for a vague goal of “best recall,” loosely tempered by precision and simplicity. This unfocused objective led to prolonged tuning efforts far beyond the point of diminishing returns.

I began with AUC as my optimization metric but realized that prioritizing recall better aligned with the business objective: minimizing false negatives (i.e., not missing employees likely to leave). This pivot dramatically improved performance, especially for Logistic Regression, which initially performed poorly on recall despite acceptable accuracy.

From a modeling standpoint, I built parallel pipelines for logistic and tree-based models. For Logistic Regression, I removed extreme tenure outliers, which introduced a slight divergence between training sets. Going forward, I standardized all data preparation using `Pipeline` objects to streamline preprocessing, reduce errors, and prevent data leakage.

I implemented stratified splits throughout training and validation to maintain class distribution. All models used cross-validation with Stratified K-Folds, and I progressively incorporated class weighting to account for imbalance.

Hyperparameter tuning began with `GridSearchCV` but shifted to `RandomizedSearchCV` for computationally intensive models like Random Forest and XGBoost. This allowed me to iterate more efficiently while maintaining model quality. Ultimately, I tuned logistic and decision tree models via grid search and the ensemble models via random search.

Feature engineering and selection were iterative, but gains eventually plateaued. To investigate further, I analyzed misclassifications from baseline models. The ambiguous nature of many false positives and negatives indicated that most misclassified employees fell into genuine gray zones—not easily distinguishable as stayers or leavers.

At this point, I finalized model selection criteria to align with both business and performance goals:

- A **minimum recall threshold** of 0.935

- A **minimum F₂ score** of 0.85 (prioritizing recall at a 4:1 weight over precision)

- Preference for models with **fewer features**

- Tie-breaking by **highest F₂**, followed by **precision**

This framework allowed me to choose the strongest models with a well-balanced approach to performance and interpretability. With model pipelines finalized and selection criteria defined, I was ready for the final evaluation on the test set (`X_test` and `X_test_lr`)—to plot the results and translate the findings into actionable insights for stakeholders.



<a id="cross-validation-results)"></a>

## Cross-Validation Results


[Back to top](#)

This section presents the cross-validated results of all modeling strategies to date. Here, we summarize model performance prior to retraining the final selected models on the full training set and evaluating them on the holdout test set (`X_test`). The results include a comprehensive table of evaluation metrics for each model, as well as visualizations of confusion matrices for both the baseline and top-performing models. These outputs provide a clear comparison of model effectiveness and support transparent, data-driven model selection.

In [6]:
# show confusion matrix exemplar
# display(Image(filename="../resources/images/confusion_matrix_exemplar.png", width=800))
display(
    HTML(
        """
<img src="../resources/images/confusion_matrix_exemplar.png" style="max-width:100%; height:auto; display:block; margin:auto;">
"""
    )
)

In [7]:
# display confusion matrix for base model
# display(Image(filename="../results/images/base_model_confusion_matrices_confusion_grid.png", width=800))
display(
    HTML(
        """
<img src="../results/images/base_model_confusion_matrices_confusion_grid.png" style="max-width:100%; height:auto; display:block; margin:auto;">
"""
    )
)

In [8]:
# display all model evaluation results
df_results = pd.read_csv("../results/all_model_evaluation_results.csv")
df_results = df_results.rename(
    columns={
        "model": "Model",
        "recall": "Recall",
        "f2": "F2 Score",
        "f1": "F1 Score",
        "roc_auc": "ROC AUC",
        "precision": "Precision",
        "accuracy": "Accuracy",
        "features": "Num Features",
        "best_params": "Best Params",
        "cv_best_score": "CV Best Score",
        "conf_matrix": "Confusion Matrix",
        "search_time": "Search Time (s)",
    }
)
df_results[
    [
        "Model",
        "Recall",
        "F2 Score",
        "F1 Score",
        "Precision",
        "Accuracy",
        "ROC AUC",
        "Num Features",
        "Confusion Matrix",
        "Search Time (s)",
    ]
]

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix,Search Time (s)
0,Logistic Regression (Core + Interactions),0.962126,0.817433,0.666974,0.510398,0.838128,0.888335,6,"[[6039, 1389], [57, 1448]]",0.801151
1,Logistic Regression with Interaction (feature ...,0.960133,0.819161,0.671312,0.516071,0.841599,0.891666,10,"[[6073, 1355], [60, 1445]]",2.056098
2,Logistic Regression (Core + Interactions + Bur...,0.951495,0.825836,0.689290,0.540377,0.855480,0.903177,6,"[[6210, 1218], [73, 1432]]",0.982459
3,Logistic Regression (base),0.947508,0.796203,0.642342,0.485860,0.822232,0.891388,18,"[[5919, 1509], [79, 1426]]",5.170930
4,Decision Tree (Core + Burnout),0.943503,0.887040,0.813972,0.715714,0.928378,0.956257,7,"[[7402, 597], [90, 1503]]",2.831705
5,Decision Tree (base),0.942247,0.881179,0.803103,0.699767,0.923269,0.945551,18,"[[7355, 644], [92, 1501]]",2.358475
6,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.753266,0.939012,0.976981,7,"[[7508, 491], [94, 1499]]",78.417716
7,Random Forest (base),0.940364,0.863799,0.769784,0.651588,0.906589,0.964169,18,"[[7198, 801], [95, 1498]]",178.203500
8,Logistic Regression with Binning (feature sele...,0.939535,0.855104,0.753531,0.629004,0.896451,0.947481,14,"[[6594, 834], [91, 1414]]",1.312480
9,Logistic Regression with Binning,0.937542,0.854944,0.755151,0.632168,0.897571,0.952159,26,"[[6607, 821], [94, 1411]]",3.109411


In [9]:
# display confusion matrix for all models, top 9
# display(Image(filename="../results/images/top_model_confusion_matrices_confusion_grid.png", width=800))
display(
    HTML(
        """
<img src="../results/images/top_model_confusion_matrices_confusion_grid.png" style="max-width:100%; height:auto; display:block; margin:auto;">
"""
    )
)

<a id="model-building"></a>

## Model Building
---

[Back to top](#)


# Plan

one section for each type of model (log reg, dt, rf, xgb)

-

-

show the df of results for versions of that type of model

give a rationale for why i'm choosing which version of the model (show confusion matrix)

run it through the cross-validation model evaluation function again and save the model, using the exact same Pipeline and random_state

train the saved model on all of X_train (save it again)

run the model on X_test

compare results of training and testing confusion matrices, plot feature importance, pr-roc, etc.

interpret the model, how it makes decisions, business implications

-

-

summarize all four versions at the end, with summary table / bullet points comparing recall, precision, F1, and interpretability

#### **Choose evaluation metric**

While **ROC AUC** is a common metric for evaluating binary classifiers—offering a threshold-independent measure of how well the model distinguishes between classes—it is **not ideal for imbalanced problems like employee churn**, where the positive class (those likely to leave) is much smaller and more critical to identify.

During model development, I did review ROC AUC to get a general sense of model discrimination. However, for **model selection and tuning**, I ultimately prioritized **recall**. A high recall ensures that we identify as many at-risk employees as possible, aligning with the company's goal to support retention through early intervention. Missing a potential churner (a false negative) is generally more costly than mistakenly flagging someone who is not at risk (a false positive), especially when interventions are supportive rather than punitive.

While precision is also important—since too many false positives could dilute resources or create unnecessary concern—recall is more aligned with a **proactive retention strategy**. This tradeoff assumes that HR interventions are constructive and that the company has systems in place to act ethically on model outputs.

To avoid unintended harm, I recommend implementing **clear usage guidelines** and **transparency** measures, ensuring that predictions are used to help employees, not penalize them. Calibration and regular fairness audits should accompany any deployment of the model.

#### **Evaluation Tie Breaker**

 One final twist (I hope). I made the classic mistake of not clearly and rigidly defining success, and I now have a bunch of models that are all excellent at recall, hovering in the 0.93-0.96 range. So I'm making a post-hoc call. At least this one, I'm planning ahead of time. The best model (of each type) will be chosen based on the following tie-breakers (in order):
- recall > 0.935
- f2 > 0.85 (f2 is a new score, weighing recall at 80%, and precision at 20%)
- fewest number of features
- highest f2
- highest precision

I should hope i can make a choice by then. there can't be that many models. I... hehehe... predict... that I'll have it by number three, fewest number of features

In [10]:
'''Set evaluation metric'''

scoring = "recall"


# for XGBoost eval_metric
def get_xgb_eval_metric(scoring):
    mapping = {
        "roc_auc": "auc",  # area under ROC curve
        "accuracy": "error",  # classification error rate
        "f1": "logloss",  # logarithmic loss (not F1, but closest available)
        "precision": "logloss",  # no direct precision metric, logloss is a common fallback
        "recall": "logloss",  # no direct recall metric, logloss is a common fallback
    }
    return mapping.get(scoring, "auc")  # default to 'auc' if not found

In [11]:
'''Encode categorical variables.'''

# copy the dataframe to avoid modifying the original
df_enc = df.copy()

# encode salary as ordinal
df_enc["salary"] = df_enc["salary"].map({"low": 0, "medium": 1, "high": 2})

# encode department as dummies
df_enc = pd.get_dummies(df_enc, columns=["department"])

# confirm the changes
# print("Original salary values:\n", df["salary"].value_counts())
# print("\nEncoded salary values:\n", df_enc["salary"].value_counts())
# df_enc.columns

In [12]:
'''
Split the data into train / test sets for different models

One set for tree-based models (decision tree, random forest, XGBoost)
Another set for logistic regression (which must have outliers removed and data normalized)
Stratify the target variable each time to account for class imbalance.
'''

# split the data into features and target variable for tree-based models
X = df_enc.drop(columns=["left"])
y = df_enc["left"]

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# scale_pos_weight for XGBoost (ratio of negative to positive class in training set)
# handles class imbalance
scale_pos_weight_value = (y_train == 0).sum() / (y_train == 1).sum()

In [13]:
# split the data into features and target variable for logistic regression
# remove outliers from tenure for logistic regression
df_enc_lr = df_enc.copy()

"""
outliers defined waaaaaay up above, 
at end of inital data exploration and cleaning
code not needed here, but copied for reference
"""
# q1 = df.tenure.quantile(0.25)
# q3 = df.tenure.quantile(0.75)
# iqr = q3 - q1
# upper_bound = q3 + 1.5 * iqr

# remove outliers
df_enc_lr = df_enc_lr[df_enc_lr.tenure < upper_bound]

X_lr = df_enc_lr.drop(columns=["left"])
y_lr = df_enc_lr["left"]

# split the data into training and testing sets for logistic regression
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(
    X_lr, y_lr, test_size=0.2, random_state=42, stratify=y_lr
)

In [14]:
'''
Supporting functions for model evaluation and feature engineering.

Functions to assemble model configurations,
run those models through cross-validation and hyperparameter tuning for evaluation,
plot confusion matrices,
plot feature importances, 
and do feature engineering.
'''

# build models_config for run_model_evaluation
def make_models_config(
    models,
    X_train,
    y_train,
    feature_func=None,  # can be a function, list, dict, or None
    param_grids=None,
    scaler=None,
    name_suffix="",
):
    """
    Build models_config for run_model_evaluation.
    - models: dict of {name: estimator}
    - X_train, y_train: training data
    - feature_func: function, list of functions, dict of {name: func}, or None
    - param_grids: dict of {name: param_grid} (or None for empty)
    - scaler: sklearn transformer (e.g., StandardScaler) or None
    - name_suffix: string to append to model name
    """
    configs = []
    for name, model in models.items():
        # order of steps matters, features first, then scaler, then model
        steps = []

        # determine which feature_func to use for this model
        func = None
        if isinstance(feature_func, dict):  # dict of {name: func}
            func = feature_func.get(name)
        elif callable(feature_func) or isinstance(feature_func, list):
            func = feature_func
        # handles a list of feature functions (apply in sequence), or a single function
        if func is not None:
            if isinstance(func, list):
                for i, f in enumerate(func):
                    steps.append((f"features_{i+1}", FunctionTransformer(f)))
            else:
                steps.append(("features", FunctionTransformer(func)))

        # add scaler if provided
        if scaler is not None:
            steps.append(("scaler", scaler))

        # add model
        steps.append(("model", model))

        # create the pipeline
        pipe = Pipeline(steps)

        # add parameter grid if provided
        param_grid = {}
        if isinstance(param_grids, dict):
            param_grid = param_grids.get(name, {})

        # add model configuration to the list
        configs.append(
            {
                "name": f"{name}{name_suffix}",
                "X_train": X_train,
                "y_train": y_train,
                "pipeline": pipe,
                "param_grid": param_grid,
            }
        )
    return configs

In [15]:
# run model evaluation function
def run_model_evaluation(
    models_config,
    results_df=None,
    scoring="recall",
    save_model=False,
    search_type="grid",
    n_iter=20,
):
    """
    Run model training and evaluation for a list of model configurations using cross-validated hyperparameter search.

    For each model configuration, performs hyperparameter tuning (GridSearchCV or RandomizedSearchCV),
    fits the best pipeline, evaluates cross-validated performance metrics, and optionally saves the best model.

    Parameters:
        models_config (list of dict): List of model configurations, each containing:
            - 'name': Model name (str)
            - 'X_train': Training features (pd.DataFrame or np.ndarray)
            - 'y_train': Training labels (pd.Series or np.ndarray)
            - 'pipeline': sklearn Pipeline object
            - 'param_grid': dict of hyperparameters for search
        results_df (pd.DataFrame or None): Existing results DataFrame to append to, or None to create a new one.
        scoring (str): Scoring metric for model selection (e.g., 'recall', 'accuracy', 'roc_auc').
        save_model (bool): If True, saves the best model pipeline to disk for each configuration.
        search_type (str): 'grid' for GridSearchCV, 'random' for RandomizedSearchCV.
        n_iter (int): Number of parameter settings sampled for RandomizedSearchCV (ignored for grid search).

    Returns:
        pd.DataFrame: Results DataFrame with model name, metrics (recall, f1, roc_auc, precision, accuracy),
                      number of features, best hyperparameters, best CV score, confusion matrix, and search time.

    Notes:
        - Uses stratified 5-fold cross-validation for both hyperparameter search and out-of-fold predictions.
        - Calculates metrics on cross-validated predictions for robust performance estimates.
        - Handles models that do not support predict_proba for ROC AUC gracefully.
        - Saves models to '../results/saved_models/' if save_model=True.
    """
    if results_df is None:
        results_df = pd.DataFrame(
            columns=[
                "model",
                "recall",
                "f2",  # 80% recall, 20% precision (metric created to weigh recall more heavily)
                "f1",  # 50% recall, 50% precision
                "roc_auc",
                "precision",
                "accuracy",
                "features",
                "best_params",
                "cv_best_score",
                "conf_matrix",
                "search_time",
            ]
        )

    # ensure cross-validation is stratified for balanced class distribution
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for cfg in models_config:
        # time the model training and evaluation
        start_time = time.time()
        print(f"Running model: {cfg['name']}...")

        # conditional to choose search type, instantiate the appropriate search class
        if search_type == "random":
            grid = RandomizedSearchCV(
                cfg["pipeline"],
                cfg["param_grid"],
                n_iter=n_iter,
                cv=cv,
                scoring=scoring,
                n_jobs=-1,
                verbose=2,
                random_state=42,
            )
        else:
            grid = GridSearchCV(
                cfg["pipeline"],
                cfg["param_grid"],
                cv=cv,
                scoring=scoring,
                n_jobs=-1,
                verbose=2,
            )

        # fit the grid search to the training data
        grid.fit(cfg["X_train"], cfg["y_train"])

        # print the execution time
        end_time = time.time()
        search_time = end_time - start_time
        print(f"Execution time for {cfg['name']}: {search_time:.2f} seconds")

        # get the best model and its parameters
        best_model = grid.best_estimator_
        print(f"Best parameters for {cfg['name']}: {grid.best_params_}")
        print(f"Best score for {cfg['name']}: {grid.best_score_:.4f} ({scoring})")

        # --- get the number of features after all pipeline steps ---
        # try to transform X_train through all steps except the final estimator
        try:
            if hasattr(best_model, "named_steps"):
                # Remove the final estimator step
                steps = list(best_model.named_steps.items())
                if len(steps) > 1:
                    # Remove last step (the model)
                    feature_pipeline = Pipeline(steps[:-1])
                    X_transformed = feature_pipeline.transform(cfg["X_train"])
                    n_features = X_transformed.shape[1]
                else:
                    n_features = cfg["X_train"].shape[1]
            else:
                n_features = cfg["X_train"].shape[1]
        except Exception as e:
            print(f"Could not determine number of features: {e}")
            n_features = cfg["X_train"].shape[1]

        # conditional to save the best model
        if save_model:
            model_path = f"../results/saved_models/{cfg['name'].replace(' ', '_').lower()}.joblib"
            joblib.dump(best_model, model_path)
            print(f"Model {cfg['name']} saved successfully.\n")
        else:
            print(f"Model {cfg['name']} not saved. Set save_model=True to save it.\n")

        # make predictions using cross-validation to generate out-of-fold predictions for each training sample
        # translation:
        # substitute for setting aside a validation set
        # takes more time, but provides better estimates of model performance
        # it makes a prediction for each sample in the training set, using a different fold of the data for each prediction...
        # ...the fold where the sample is not included in the 80% training set (the sample is in the 20%)
        y_pred = cross_val_predict(
            best_model, cfg["X_train"], cfg["y_train"], cv=cv, n_jobs=-1
        )

        # # check misclassified cases for further analysis
        # print(f"Misclassified cases for {cfg['name']}:")
        # misclassified = cfg['X_train'].copy()
        # misclassified['actual'] = cfg["y_train"]
        # misclassified['predicted'] = y_pred
        # misclassified = misclassified[misclassified['actual'] != misclassified['predicted']]

        # # Show counts of each type of misclassification
        # counts = misclassified.groupby(['actual', 'predicted']).size().rename('count')
        # print("\nMisclassification counts:")
        # print(counts)
        # print()

        # # Show .describe() for each group, side by side
        # pd.set_option('display.max_columns', None)
        # for (actual, predicted), group in misclassified.groupby(['actual', 'predicted']):
        #     label_map = {0: "Stayed", 1: "Left"}
        #     print(f"--- Misclassified: Actual={label_map.get(actual, actual)}, Predicted={label_map.get(predicted, predicted)} (n={len(group)}) ---")
        #     print(group.describe().T)
        #     print()
        # pd.reset_option('display.max_columns')
        # print("\n")

        # calculate the ROC AUC score, need predicted probabilities (not just class labels, but confidence in those labels)
        # try / except block to handle models that do not support predict_proba (e.g., SVC)
        try:
            y_proba = cross_val_predict(
                best_model,
                cfg["X_train"],
                cfg["y_train"],
                cv=cv,
                method="predict_proba",
                n_jobs=-1,
            )
            roc_auc = roc_auc_score(cfg["y_train"], y_proba[:, 1])
        except (AttributeError, ValueError):
            roc_auc = np.nan
            print(f"Model {cfg['name']} does not support predict_proba.")

        # save results in the results dataframe
        results_df.loc[len(results_df)] = {
            "model": cfg["name"],
            "features": n_features,
            "accuracy": accuracy_score(cfg["y_train"], y_pred),
            "precision": precision_score(cfg["y_train"], y_pred),
            "recall": recall_score(cfg["y_train"], y_pred),
            "f1": f1_score(cfg["y_train"], y_pred),
            "f2": fbeta_score(
                cfg["y_train"], y_pred, beta=2
            ),  # 80% recall, 20% precision (ratio is "beta squared : 1", b^2:1, 2^2:1, 4:1)
            "roc_auc": roc_auc,
            "conf_matrix": confusion_matrix(cfg["y_train"], y_pred).tolist(),
            "best_params": grid.best_params_,
            "cv_best_score": grid.best_score_,
            "search_time": search_time,
        }

    return results_df

In [16]:
# plot confusion matrices from dataframe
def plot_confusion_from_results(results_df, save_png=False):
    """Plots SINGLE confusion matrices from results dataframe and optionally saves png."""

    class_labels = ["Stayed", "Left"]

    for idx, row in results_df.iterrows():
        cm = row["conf_matrix"]
        model_name = row["model"]

        plt.figure(figsize=(5, 4))
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            cbar=False,
            xticklabels=class_labels,
            yticklabels=class_labels,
        )
        plt.title(f"Confusion Matrix: {model_name}")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.tight_layout()

        # conditional to save the confusion matrix as a PNG file
        if save_png:
            plt.savefig(
                f"../results/images/{model_name.replace(' ', '_').lower()}_confusion_matrix.png"
            )

        plt.show()

In [17]:
# plot confusion matrix grid from dataframe
def plot_confusion_grid_from_results(results_df, png_title=None):
    """Plots ALL confusion matrices from results_df IN A GRID and optionally saves png."""
    import math

    class_labels = ["Stayed", "Left"]
    n_models = len(results_df)
    n_cols = 2 if n_models <= 4 else 3
    n_rows = math.ceil(n_models / n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))
    axes = axes.flatten() if n_models > 1 else [axes]

    for idx, (i, row) in enumerate(results_df.iterrows()):
        cm = row["conf_matrix"]
        model_name = row["model"]
        ax = axes[idx]
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            cbar=False,
            xticklabels=class_labels,
            yticklabels=class_labels,
            ax=ax,
        )
        ax.set_title(f"{model_name}")
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")

    # hide any unused subplots
    for j in range(idx + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()

    # conditional to save the confusion grid as a PNG file
    if png_title:
        fig.suptitle(png_title, fontsize=16, y=1.03)
        fig.savefig(
            f"../results/images/{png_title.replace(' ', '_').lower()}_confusion_grid.png",
            bbox_inches="tight",
        )

    plt.show()

In [18]:
# plot feature importances function
def load_and_plot_feature_importance(
    file_name, model_name, feature_names, top_n=10, save_png=False
):
    """Load a model and plot its feature importance, optionally saves png."""

    # load model
    model_path = os.path.join("../results/saved_models", file_name)
    model = joblib.load(model_path)

    # if model is a pipeline, get the estimator
    if hasattr(model, "named_steps"):
        # for logistic regression, get the scaler's feature names if available
        # NOTE: StandardScaler does not change feature names, so X_train_lr.columns is correct here
        # if using a transformer that changes the feature set (e.g., OneHotEncoder, ColumnTransformer)...
        # ...one would need to extract the transformed feature names from the transformer
        estimator = model.named_steps["model"]
    # if model is not a pipeline, use it directly (irrelevant for this case, but included for future-proofing)
    else:
        estimator = model

    # get importances
    # for tree-based models, use feature_importances_ or coef_ for logistic regression
    if hasattr(estimator, "feature_importances_"):
        importances = estimator.feature_importances_
        title = "Feature Importance"
    elif hasattr(estimator, "coef_"):
        importances = np.abs(estimator.coef_[0])
        title = "Absolute Coefficient Magnitude"
    else:
        print(f"Model {model_name} does not support feature importance.")
        return

    # sort and select top N
    indices = np.argsort(importances)[::-1][:top_n]

    plt.figure(figsize=(8, 5))
    plt.barh(np.array(feature_names)[indices][::-1], importances[indices][::-1])
    plt.xlabel(title)
    plt.title(f"{model_name}: Top {top_n} Features")
    plt.tight_layout()

    # conditional to save the feature importance plot as a PNG file
    if save_png:
        plt.savefig(
            f"../results/images/{model_name.replace(' ', '_').lower()}_feature_importance.png"
        )

    plt.show()

In [19]:
'''
# feature engineering / feature selection functions
# the functions commented out below are not used in the final notebook,
# but were used during model development
# and are included for reference
'''

# NOTE used in Logistic Regression's winner - "with Binning (feature selection)"
# add binning features
def add_binning_features(df):
    df = df.copy()
    df["satisfaction_bin"] = pd.cut(
        df["satisfaction_level"],
        bins=[-0.01, 0.4, 0.7, 1.0],
        labels=["low", "medium", "high"],
    )
    df["hours_bin"] = pd.cut(
        df["average_monthly_hours"],
        bins=[0, 160, 240, np.inf],
        labels=["low", "medium", "high"],
    )
    df["projects_bin"] = pd.cut(
        df["number_project"], bins=[0, 2, 5, np.inf], labels=["low", "medium", "high"]
    )
    df["tenure_bin"] = pd.cut(
        df["tenure"], bins=[0, 3, 5, np.inf], labels=["short", "mid", "long"]
    )
    # encode the binned features as dummies
    df = pd.get_dummies(
        df,
        columns=["satisfaction_bin", "hours_bin", "projects_bin", "tenure_bin"],
        drop_first=True,
    )
    return df


# # add interaction features
# def add_interaction_features(df):
#     df = df.copy()
#     df["satisfaction_x_projects"] = df["satisfaction_level"] * df["number_project"]
#     df["satisfaction_x_hours"] = df["satisfaction_level"] * df["average_monthly_hours"]
#     df["evaluation_x_satisfaction"] = df["last_evaluation"] * df["satisfaction_level"]
#     df["hours_per_project"] = df["average_monthly_hours"] / df["number_project"]
#     return df


# # add flag features
# def add_flag_features(df):
#     df = df.copy()
#     df["burnout"] = (
#         (df["number_project"] >= 6) | (df["average_monthly_hours"] >= 240)
#     ) & (df["satisfaction_level"] <= 0.3)
#     df["disengaged"] = (
#         (df["number_project"] <= 2)
#         & (df["average_monthly_hours"] < 160)
#         & (df["satisfaction_level"] <= 0.5)
#     )
#     df["no_promo_4yr"] = (df["promotion_last_5years"] == 0) & (df["tenure"] >= 4)
#     return df

# NOTE used in Logistic Regression's winner - "with Binning (feature selection)"
# feature selection for logistic regression
drop_cols = [col for col in X_train_lr.columns if col.startswith("department_")]
drop_cols += ["salary", "work_accident"]
X_train_lr_fs = X_train_lr.drop(columns=drop_cols)

# # feature selection for tree-based models
# drop_cols = [col for col in X_train.columns if col.startswith("department_")]
# drop_cols += ["salary", "work_accident"]
# X_train_fs = X_train.drop(columns=drop_cols)

In [20]:
# NOTE This is the group of features that 'won' for all tree-based models
# selected features + burnout flag
def select_core_features_with_burnout(df):
    df = df.copy()
    # burnout flag: (projects >= 6 or hours >= 240) & satisfaction <= 0.3
    df["burnout"] = (
        (df["number_project"] >= 6) | (df["average_monthly_hours"] >= 240)
    ) & (df["satisfaction_level"] <= 0.3)
    return df[
        [
            "satisfaction_level",
            "last_evaluation",
            "number_project",
            "average_monthly_hours",
            "tenure",
            "promotion_last_5years",
            "burnout",
        ]
    ]


# # selected features + interactions
# def select_core_features_with_interactions(df):
#     df = df.copy()
#     # interactions
#     df["satisfaction_x_projects"] = df["satisfaction_level"] * df["number_project"]
#     df["hours_per_project"] = df["average_monthly_hours"] / df["number_project"]
#     return df[
#         [
#             "satisfaction_level",
#             "number_project",
#             "average_monthly_hours",
#             "tenure",
#             "satisfaction_x_projects",
#             "hours_per_project",
#         ]
#     ]


# # selected features + interactions + burnout flag
# def select_core_features_with_interactions_and_burnout(df):
#     df = df.copy()
#     # burnout flag: (projects >= 6 or hours >= 240) & satisfaction <= 0.3
#     df["burnout"] = (
#         (df["number_project"] >= 6) | (df["average_monthly_hours"] >= 240)
#     ) & (df["satisfaction_level"] <= 0.3)
#     # interaction
#     df["satisfaction_x_projects"] = df["satisfaction_level"] * df["number_project"]
#     return df[
#         [
#             "satisfaction_level",
#             "number_project",
#             "average_monthly_hours",
#             "tenure",
#             "burnout",
#             "satisfaction_x_projects",
#         ]
#     ]

<p class="center-icons">
  <i class="fa fa-circle spacer-icons"></i>
  <i class="fa fa-circle spacer-icons"></i>
  <i class="fa fa-circle spacer-icons"></i>
</p>

<a id="logistic-regression"></a>

## Logistic Regression

[Back to top](#)

In [21]:
# logistic regression feature engineering hyperparameters
lr_fe_params = {
    "model__C": [0.1, 1.0, 10.0],  # regularization strength (inverse)
    "model__penalty": ["l1", "l2"],  # regularization type (L1 = Lasso, L2 = Ridge)
    "model__solver": ["liblinear"],  # optimization algorithm (liblinear supports L1/L2)
    "model__class_weight": [None, "balanced"],  # None or balanced for class imbalance
}

# define feature engineered logistic regression models, their feature functions, and parameter grids
lr_fe_models = {
    "Logistic Regression with Binning": LogisticRegression(
        max_iter=1000, random_state=42
    ),
}
lr_fe_feature_funcs = {
    "Logistic Regression with Binning": add_binning_features,
}
lr_fe_param_grids = {
    "Logistic Regression with Binning": lr_fe_params,
}

# create models_config for logistic regression with feature engineering and feature selection
lr_fe_fs_configs = make_models_config(
    lr_fe_models,
    X_train_lr_fs,
    y_train_lr,
    feature_func=lr_fe_feature_funcs,
    scaler=StandardScaler(),
    param_grids=lr_fe_param_grids,
    name_suffix=" (feature selection)",
)

<div class="emphasis-box">
  <strong>Model selection criteria aligned with both business and performance goals (in order):</strong>
  <ul>
    <li>A <strong>minimum recall threshold</strong> of 0.935</li>
    <li>A <strong>minimum F<sub>2</sub> score</strong> of 0.85 (prioritizing recall at a 4:1 weight over precision)</li>
    <li>Preference for models with <strong>fewer features</strong></li>
    <li>Tie-breaking by <strong>highest F<sub>2</sub></strong>, followed by <strong>precision</strong></li>
  </ul>
</div>

All Logistic Regression Models:

In [22]:
# display model evaluation results for all logistic regression models
df_lr = df_results[df_results["Model"].str.contains("Logistic Regression")]
if df_lr.empty:
    print("No logistic regression models found in results.")

# print("Logistic Regression Model Evaluation Results:")
df_lr = df_lr[
    [
        "Model",
        "Recall",
        "F2 Score",
        "F1 Score",
        "Precision",
        "Accuracy",
        "ROC AUC",
        "Num Features",
        "Confusion Matrix",
    ]
]
df_lr

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
0,Logistic Regression (Core + Interactions),0.962126,0.817433,0.666974,0.510398,0.838128,0.888335,6,"[[6039, 1389], [57, 1448]]"
1,Logistic Regression with Interaction (feature ...,0.960133,0.819161,0.671312,0.516071,0.841599,0.891666,10,"[[6073, 1355], [60, 1445]]"
2,Logistic Regression (Core + Interactions + Bur...,0.951495,0.825836,0.689290,0.540377,0.855480,0.903177,6,"[[6210, 1218], [73, 1432]]"
3,Logistic Regression (base),0.947508,0.796203,0.642342,0.485860,0.822232,0.891388,18,"[[5919, 1509], [79, 1426]]"
8,Logistic Regression with Binning (feature sele...,0.939535,0.855104,0.753531,0.629004,0.896451,0.947481,14,"[[6594, 834], [91, 1414]]"
9,Logistic Regression with Binning,0.937542,0.854944,0.755151,0.632168,0.897571,0.952159,26,"[[6607, 821], [94, 1411]]"
16,Logistic Regression (Core + Burnout),0.934884,0.798796,0.655638,0.504844,0.834546,0.886564,7,"[[6048, 1380], [98, 1407]]"
20,Logistic Regression with Interaction,0.933555,0.808121,0.672571,0.525627,0.846860,0.901327,22,"[[6160, 1268], [100, 1405]]"
38,Logistic Regression with Flags (feature select...,0.917608,0.870524,0.808311,0.722280,0.926676,0.953360,9,"[[6897, 531], [124, 1381]]"
39,Logistic Regression with Flags,0.917608,0.868335,0.803608,0.714803,0.924437,0.957952,21,"[[6877, 551], [124, 1381]]"


Above the minimum recall threshold of 0.935:

In [23]:
df_lr_1 = df_lr[df_lr["Recall"] >= 0.935]
df_lr_1

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
0,Logistic Regression (Core + Interactions),0.962126,0.817433,0.666974,0.510398,0.838128,0.888335,6,"[[6039, 1389], [57, 1448]]"
1,Logistic Regression with Interaction (feature ...,0.960133,0.819161,0.671312,0.516071,0.841599,0.891666,10,"[[6073, 1355], [60, 1445]]"
2,Logistic Regression (Core + Interactions + Bur...,0.951495,0.825836,0.689290,0.540377,0.855480,0.903177,6,"[[6210, 1218], [73, 1432]]"
3,Logistic Regression (base),0.947508,0.796203,0.642342,0.485860,0.822232,0.891388,18,"[[5919, 1509], [79, 1426]]"
8,Logistic Regression with Binning (feature sele...,0.939535,0.855104,0.753531,0.629004,0.896451,0.947481,14,"[[6594, 834], [91, 1414]]"
9,Logistic Regression with Binning,0.937542,0.854944,0.755151,0.632168,0.897571,0.952159,26,"[[6607, 821], [94, 1411]]"


Above the minimum F₂ threshold of 0.85:

In [24]:
df_lr_2 = df_lr_1[df_lr_1["F2 Score"] >= 0.85]
df_lr_2

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
8,Logistic Regression with Binning (feature sele...,0.939535,0.855104,0.753531,0.629004,0.896451,0.947481,14,"[[6594, 834], [91, 1414]]"
9,Logistic Regression with Binning,0.937542,0.854944,0.755151,0.632168,0.897571,0.952159,26,"[[6607, 821], [94, 1411]]"


With the fewest features:

In [25]:
df_lr_winner = df_lr_2[df_lr_2["Num Features"] == df_lr_2["Num Features"].min()]
df_lr_winner

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
8,Logistic Regression with Binning (feature sele...,0.939535,0.855104,0.753531,0.629004,0.896451,0.947481,14,"[[6594, 834], [91, 1414]]"


And the winner is "Logistic Regression with Binning (feature selection)"

In [26]:
''' 
Model, feature function, and parameter grid definitions
 (used to build pipelines) 
 located at beginning of Logistic Regression section 
 '''
results_training = run_model_evaluation(lr_fe_fs_configs, scoring=scoring)

Running model: Logistic Regression with Binning (feature selection)...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Execution time for Logistic Regression with Binning (feature selection): 6.32 seconds
Best parameters for Logistic Regression with Binning (feature selection): {'model__C': 0.1, 'model__class_weight': 'balanced', 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Best score for Logistic Regression with Binning (feature selection): 0.9395 (recall)
Model Logistic Regression with Binning (feature selection) not saved. Set save_model=True to save it.



#### Data Dictionary: Logistic Regression with Binning (Feature Selection)

This model uses a subset of the original features, applies binning to key variables, and drops department, salary, and work_accident.

| Variable                       | Description                                                                                 |
|---------------------------------|---------------------------------------------------------------------------------------------|
| satisfaction_level              | Employee-reported job satisfaction level [0–1]                                              |
| last_evaluation                 | Score of employee's last performance review [0–1]                                           |
| number_project                  | Number of projects employee contributes to                                                  |
| average_monthly_hours           | Average number of hours employee worked per month                                           |
| tenure                          | How long the employee has been with the company (years)                                     |
| promotion_last_5years           | Whether or not the employee was promoted in the last 5 years                                |
| satisfaction_bin_medium         | Binary indicator: satisfaction_level is medium (> 0.4 and ≤ 0.7)                            |
| satisfaction_bin_high           | Binary indicator: satisfaction_level is high (> 0.7)                                       |
| hours_bin_medium                | Binary indicator: average_monthly_hours is medium (> 160 and ≤ 240)                         |
| hours_bin_high                  | Binary indicator: average_monthly_hours is high (> 240)                                    |
| projects_bin_medium             | Binary indicator: number_project is medium (> 2 and ≤ 5)                                    |
| projects_bin_high               | Binary indicator: number_project is high (> 5)                                             |
| tenure_bin_mid                  | Binary indicator: tenure is mid (> 3 and ≤ 5 years)                                        |
| tenure_bin_long                 | Binary indicator: tenure is long (> 5 years)                                               |

- All binned features are one-hot encoded, with the first category dropped (i.e., "low" or "short" is the reference).

Best hyperparameters for **Logistic Regression with Binning (feature selection)**:

```python
{
    'model__C': 0.1,                     # Strong L1 regularization (encourages sparsity/feature selection)
    'model__class_weight': 'balanced',   # Adjusts for class imbalance
    'model__penalty': 'l1',              # L1 penalty (Lasso)
    'model__solver': 'liblinear'         # Solver supporting L1/L2 penalties
}
```

---

In [27]:
# compare results_training with df_dt_winner
confirm = pd.concat(
    [
        df_lr_winner,
        df_results[
            df_results["Model"].str.contains(
                "Logistic Regression with Binning (feature selection)", regex=False
            )
        ],
    ]
)
confirm

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix,Best Params,CV Best Score,Search Time (s)
8,Logistic Regression with Binning (feature sele...,0.939535,0.855104,0.753531,0.629004,0.896451,0.947481,14,"[[6594, 834], [91, 1414]]",NaN,NaN,NaN
8,Logistic Regression with Binning (feature sele...,0.939535,0.855104,0.753531,0.629004,0.896451,0.947481,14,"[[6594, 834], [91, 1414]]","{'model__C': 0.1, 'model__class_weight': 'bala...",0.939535,1.31248


<a id="decision-tree"></a>

## Decision Tree

[Back to top](#)

In [28]:
# # tree-based feature engineering parameters
tree_fe_params = {
    "Random Forest": {
        "model__n_estimators": [100, 300],  # 300 was best, but 100 is faster for FE
        "model__max_depth": [
            3,
            4,
            5,
            8,
        ],  # 5 was best, trying for regularization, deeper trees can overfit, take longer to train
        "model__max_features": ["sqrt", 1.0],  # 1.0 was best, but sqrt is common
        "model__max_samples": [0.7, 1.0],  # 1.0 was best
        "model__min_samples_leaf": [1, 2],  # 1 or 2
        "model__min_samples_split": [2, 3],  # 2 or 3
        "model__class_weight": [
            None,
            "balanced",
        ],  # None or balanced for class imbalance
    },
    "XGBoost": {
        "model__n_estimators": [100, 300],  # 300 was best
        "model__max_depth": [
            3,
            4,
            5,
            8,
        ],  # 3 was best (moderate increase in training time)
        "model__learning_rate": [
            0.1,
            0.2,
        ],  # 0.1 is standard, 0.2 for speed, step size shrinkage
        "model__subsample": [
            0.6,
            0.8,
            1.0,
        ],  # 1.0 was best, row subsampling (adds randomness, helps generalization)
        "model__colsample_bytree": [
            0.6,
            0.8,
            1.0,
        ],  # 1.0 was best, column subsampling (adds randomness, helps generalization)
        "model__min_child_weight": [
            1,
            5,
        ],  # 1 is default, 5 for regularization, minimum sum of instance weight in a child
        "model__gamma": [
            0,
            0.1,
            0.2,
        ],  # 0.2 was best, try 0 for comparison, minimum loss reduction required to make a split
        "model__scale_pos_weight": [
            1,
            scale_pos_weight_value,
        ],  # 1 or calculated value for class imbalance
        "model__reg_alpha": [
            0,
            0.1,
            1,
        ],  # L1 regularization (helps control overfitting)
        "model__reg_lambda": [1, 2, 5],  # L2 regularization (helps control overfitting)
    },
    "Decision Tree": {
        "model__max_depth": [3, 4, 5, 6, 8],  # best was 8
        "model__min_samples_leaf": [1, 2, 3],  # 1 was best
        "model__min_samples_split": [2, 3, 4],  # 2 was best
        "model__class_weight": [
            None,
            "balanced",
        ],  # None or balanced for class imbalance
    },
}

In [29]:
# winning tree-based models, feature funcs, param grids
dt_fe2_models = {
    "Decision Tree (Core + Burnout)": DecisionTreeClassifier(random_state=42),
}
rf_xgb_fe2_models = {
    "Random Forest (Core + Burnout)": RandomForestClassifier(
        random_state=42, n_jobs=-1
    ),
    "XGBoost (Core + Burnout)": XGBClassifier(
        eval_metric=get_xgb_eval_metric(scoring), random_state=42, n_jobs=-1
    ),
}
tree_fe2_feature_funcs = {
    "Decision Tree (Core + Burnout)": select_core_features_with_burnout,
    "Random Forest (Core + Burnout)": select_core_features_with_burnout,
    "XGBoost (Core + Burnout)": select_core_features_with_burnout,
}
tree_fe2_param_grids = {
    "Decision Tree (Core + Burnout)": tree_fe_params["Decision Tree"],
    "Random Forest (Core + Burnout)": tree_fe_params["Random Forest"],
    "XGBoost (Core + Burnout)": tree_fe_params["XGBoost"],
}

# create models_config for decision tree and random forest / XGBoost with feature engineering and feature selection
dt_fe2_configs = make_models_config(
    dt_fe2_models,
    X_train,
    y_train,
    feature_func=tree_fe2_feature_funcs,
    param_grids=tree_fe2_param_grids,
)
rf_xgb_fe2_configs = make_models_config(
    rf_xgb_fe2_models,
    X_train,
    y_train,
    feature_func=tree_fe2_feature_funcs,
    param_grids=tree_fe2_param_grids,
)

<div class="emphasis-box">
  <strong>Model selection criteria aligned with both business and performance goals (in order):</strong>
  <ul>
    <li>A <strong>minimum recall threshold</strong> of 0.935</li>
    <li>A <strong>minimum F<sub>2</sub> score</strong> of 0.85 (prioritizing recall at a 4:1 weight over precision)</li>
    <li>Preference for models with <strong>fewer features</strong></li>
    <li>Tie-breaking by <strong>highest F<sub>2</sub></strong>, followed by <strong>precision</strong></li>
  </ul>
</div>

All Decision Tree Models:

In [30]:
# display model evaluation results for all decision tree models
df_dt = df_results[df_results["Model"].str.contains("Decision Tree")]
if df_dt.empty:
    print("No decision tree models found in results.")

# print("Decision Tree Model Evaluation Results:")
df_dt = df_dt[
    [
        "Model",
        "Recall",
        "F2 Score",
        "F1 Score",
        "Precision",
        "Accuracy",
        "ROC AUC",
        "Num Features",
        "Confusion Matrix",
    ]
]
df_dt

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
4,Decision Tree (Core + Burnout),0.943503,0.887040,0.813972,0.715714,0.928378,0.956257,7,"[[7402, 597], [90, 1503]]"
5,Decision Tree (base),0.942247,0.881179,0.803103,0.699767,0.923269,0.945551,18,"[[7355, 644], [92, 1501]]"
15,Decision Tree with Binning (feature selection),0.935342,0.920776,0.899758,0.866783,0.965388,0.959523,14,"[[7770, 229], [103, 1490]]"
18,Decision Tree with Binning,0.934087,0.918632,0.896386,0.861610,0.964137,0.956223,26,"[[7760, 239], [105, 1488]]"
21,Decision Tree with Interaction,0.933459,0.907150,0.870354,0.815241,0.953816,0.960404,22,"[[7662, 337], [106, 1487]]"
22,Decision Tree with Interaction (feature select...,0.932831,0.902795,0.861200,0.799785,0.950063,0.957669,10,"[[7627, 372], [107, 1486]]"
25,Decision Tree (Core + Interactions),0.931576,0.909202,0.877587,0.829514,0.956839,0.960284,6,"[[7694, 305], [109, 1484]]"
27,Decision Tree with Flags,0.930948,0.909035,0.878034,0.830812,0.957048,0.966076,21,"[[7697, 302], [110, 1483]]"
29,Decision Tree (Core + Interactions + Burnout),0.930320,0.897420,0.852214,0.786207,0.946414,0.955136,6,"[[7596, 403], [111, 1482]]"
31,Decision Tree with Flags (feature selection),0.929692,0.909705,0.881285,0.837670,0.958403,0.965284,9,"[[7712, 287], [112, 1481]]"


Above the minimum recall threshold of 0.935:

In [31]:
df_dt_1 = df_dt[df_dt["Recall"] >= 0.935]
df_dt_1

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
4,Decision Tree (Core + Burnout),0.943503,0.887040,0.813972,0.715714,0.928378,0.956257,7,"[[7402, 597], [90, 1503]]"
5,Decision Tree (base),0.942247,0.881179,0.803103,0.699767,0.923269,0.945551,18,"[[7355, 644], [92, 1501]]"
15,Decision Tree with Binning (feature selection),0.935342,0.920776,0.899758,0.866783,0.965388,0.959523,14,"[[7770, 229], [103, 1490]]"


Above the minimum F₂ threshold of 0.85:

In [32]:
df_dt_2 = df_dt_1[df_dt_1["F2 Score"] >= 0.85]
df_dt_2

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
4,Decision Tree (Core + Burnout),0.943503,0.887040,0.813972,0.715714,0.928378,0.956257,7,"[[7402, 597], [90, 1503]]"
5,Decision Tree (base),0.942247,0.881179,0.803103,0.699767,0.923269,0.945551,18,"[[7355, 644], [92, 1501]]"
15,Decision Tree with Binning (feature selection),0.935342,0.920776,0.899758,0.866783,0.965388,0.959523,14,"[[7770, 229], [103, 1490]]"


With the fewest features:

In [33]:
df_dt_winner = df_dt_2[df_dt_2["Num Features"] == df_dt_2["Num Features"].min()]
df_dt_winner

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
4,Decision Tree (Core + Burnout),0.943503,0.88704,0.813972,0.715714,0.928378,0.956257,7,"[[7402, 597], [90, 1503]]"


And the winner is "Decision Tree (Core + Burnout)"

In [34]:
''' 
Model, feature function, and parameter grid definitions for winning tree-based models
 (used to build pipelines) 
 located at beginning of Decision Tree section 
 '''
results_training = run_model_evaluation(
    dt_fe2_configs, results_df=results_training, scoring=scoring
)
results_training[results_training["model"] == "Decision Tree (Core + Burnout)"]

Running model: Decision Tree (Core + Burnout)...
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Execution time for Decision Tree (Core + Burnout): 3.03 seconds
Best parameters for Decision Tree (Core + Burnout): {'model__class_weight': 'balanced', 'model__max_depth': 5, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2}
Best score for Decision Tree (Core + Burnout): 0.9435 (recall)
Model Decision Tree (Core + Burnout) not saved. Set save_model=True to save it.



,model,recall,f2,f1,roc_auc,precision,accuracy,features,best_params,cv_best_score,conf_matrix,search_time
1,Decision Tree (Core + Burnout),0.943503,0.88704,0.813972,0.956257,0.715714,0.928378,7,"{'model__class_weight': 'balanced', 'model__ma...",0.943495,"[[7402, 597], [90, 1503]]",3.033708


#### Data Dictionary: Decision Tree — "Core + Burnout" Features

This model uses a focused set of original features plus an engineered "burnout" flag.

| Variable                | Description                                                                                       |
|-------------------------|---------------------------------------------------------------------------------------------------|
| satisfaction_level      | Employee-reported job satisfaction level [0–1]                                                    |
| last_evaluation         | Score of employee's last performance review [0–1]                                                 |
| number_project          | Number of projects employee contributes to                                                        |
| average_monthly_hours   | Average number of hours employee worked per month                                                 |
| tenure                  | How long the employee has been with the company (years)                                           |
| promotion_last_5years   | Whether or not the employee was promoted in the last 5 years                                      |
| burnout                 | Flag: True if (number_project ≥ 6 or average_monthly_hours ≥ 240) and satisfaction_level ≤ 0.3    |

- The "burnout" feature is a logical flag engineered to capture high-risk, overworked, and dissatisfied employees.

Best hyperparameters for **Decision Tree (Core + Burnout)**:

```python
{
    'model__class_weight': 'balanced',   # Adjusts for class imbalance
    'model__max_depth': 5,               # Shallow tree to prevent overfitting
    'model__min_samples_leaf': 1,        # Minimum samples per leaf node
    'model__min_samples_split': 2        # Minimum samples to split an internal node
}
```

---

In [35]:
# compare results_training with df_dt_winner
confirm = pd.concat(
    [
        df_dt_winner,
        df_results[
            df_results["Model"].str.contains(
                "Decision Tree (Core + Burnout)", regex=False
            )
        ],
    ]
)
confirm

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix,Best Params,CV Best Score,Search Time (s)
4,Decision Tree (Core + Burnout),0.943503,0.88704,0.813972,0.715714,0.928378,0.956257,7,"[[7402, 597], [90, 1503]]",NaN,NaN,NaN
4,Decision Tree (Core + Burnout),0.943503,0.88704,0.813972,0.715714,0.928378,0.956257,7,"[[7402, 597], [90, 1503]]","{'model__class_weight': 'balanced', 'model__ma...",0.943495,2.831705


<a id="random-forest"></a>

## Random Forest

[Back to top](#)

<div class="emphasis-box">
  <strong>Model selection criteria aligned with both business and performance goals (in order):</strong>
  <ul>
    <li>A <strong>minimum recall threshold</strong> of 0.935</li>
    <li>A <strong>minimum F<sub>2</sub> score</strong> of 0.85 (prioritizing recall at a 4:1 weight over precision)</li>
    <li>Preference for models with <strong>fewer features</strong></li>
    <li>Tie-breaking by <strong>highest F<sub>2</sub></strong>, followed by <strong>precision</strong></li>
  </ul>
</div>

All Random Forest Models:

In [36]:
# display model evaluation results for all random forest models
df_rf = df_results[df_results["Model"].str.contains("Random Forest")]
if df_rf.empty:
    print("No random forest models found in results.")

# print("Random Forest Model Evaluation Results:")
df_rf = df_rf[
    [
        "Model",
        "Recall",
        "F2 Score",
        "F1 Score",
        "Precision",
        "Accuracy",
        "ROC AUC",
        "Num Features",
        "Confusion Matrix",
    ]
]
df_rf

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
6,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.753266,0.939012,0.976981,7,"[[7508, 491], [94, 1499]]"
7,Random Forest (base),0.940364,0.863799,0.769784,0.651588,0.906589,0.964169,18,"[[7198, 801], [95, 1498]]"
28,Random Forest with Interaction (feature select...,0.930320,0.904210,0.867681,0.812946,0.952877,0.975869,10,"[[7658, 341], [111, 1482]]"
30,Random Forest with Interaction,0.929692,0.903710,0.867350,0.812843,0.952773,0.975924,22,"[[7658, 341], [112, 1481]]"
32,Random Forest (Core + Interactions + Burnout),0.929692,0.900852,0.860796,0.801407,0.950063,0.973889,6,"[[7632, 367], [112, 1481]]"
33,Random Forest with Flags (feature selection),0.929065,0.873878,0.802385,0.706107,0.923999,0.973045,9,"[[7383, 616], [113, 1480]]"
34,Random Forest with Flags,0.929065,0.873878,0.802385,0.706107,0.923999,0.973169,21,"[[7383, 616], [113, 1480]]"
35,Random Forest with Binning (feature selection),0.928437,0.923682,0.916641,0.905141,0.971956,0.974975,14,"[[7844, 155], [114, 1479]]"
36,Random Forest with Binning,0.927809,0.923173,0.916305,0.905083,0.971852,0.974756,26,"[[7844, 155], [115, 1478]]"
37,Random Forest (Core + Interactions),0.927809,0.861707,0.778509,0.670599,0.912323,0.960897,6,"[[7273, 726], [115, 1478]]"


Above the minimum recall threshold of 0.935:

In [37]:
df_rf_1 = df_rf[df_rf["Recall"] >= 0.935]
df_rf_1

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
6,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.753266,0.939012,0.976981,7,"[[7508, 491], [94, 1499]]"
7,Random Forest (base),0.940364,0.863799,0.769784,0.651588,0.906589,0.964169,18,"[[7198, 801], [95, 1498]]"


Above the minimum F₂ threshold of 0.85:

In [38]:
df_rf_2 = df_rf_1[df_rf_1["F2 Score"] >= 0.85]
df_rf_2

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
6,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.753266,0.939012,0.976981,7,"[[7508, 491], [94, 1499]]"
7,Random Forest (base),0.940364,0.863799,0.769784,0.651588,0.906589,0.964169,18,"[[7198, 801], [95, 1498]]"


With the fewest features:

In [39]:
df_rf_winner = df_rf_2[df_rf_2["Num Features"] == df_rf_2["Num Features"].min()]
df_rf_winner

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
6,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.753266,0.939012,0.976981,7,"[[7508, 491], [94, 1499]]"


And the winner is "Random Forest (Core + Burnout)"

In [40]:
''' 
Model, feature function, and parameter grid definitions for winning tree-based models
 (used to build pipelines) 
 located at beginning of Decision Tree section 
 '''
rf_config = [
    cfg for cfg in rf_xgb_fe2_configs if cfg["name"] == "Random Forest (Core + Burnout)"
]
results_training = run_model_evaluation(
    rf_config,
    results_df=results_training,
    scoring=scoring,
    search_type="random",
    n_iter=50,
)
results_training[results_training["model"] == "Random Forest (Core + Burnout)"]

Running model: Random Forest (Core + Burnout)...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Execution time for Random Forest (Core + Burnout): 71.20 seconds
Best parameters for Random Forest (Core + Burnout): {'model__n_estimators': 300, 'model__min_samples_split': 2, 'model__min_samples_leaf': 1, 'model__max_samples': 1.0, 'model__max_features': 1.0, 'model__max_depth': 5, 'model__class_weight': 'balanced'}
Best score for Random Forest (Core + Burnout): 0.9410 (recall)
Model Random Forest (Core + Burnout) not saved. Set save_model=True to save it.



,model,recall,f2,f1,roc_auc,precision,accuracy,features,best_params,cv_best_score,conf_matrix,search_time
2,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.976981,0.753266,0.939012,7,"{'model__n_estimators': 300, 'model__min_sampl...",0.940991,"[[7508, 491], [94, 1499]]",71.203241


#### Data Dictionary: Random Forest — "Core + Burnout" Features

This model uses a focused set of original features plus an engineered "burnout" flag.

| Variable                | Description                                                                                       |
|-------------------------|---------------------------------------------------------------------------------------------------|
| satisfaction_level      | Employee-reported job satisfaction level [0–1]                                                    |
| last_evaluation         | Score of employee's last performance review [0–1]                                                 |
| number_project          | Number of projects employee contributes to                                                        |
| average_monthly_hours   | Average number of hours employee worked per month                                                 |
| tenure                  | How long the employee has been with the company (years)                                           |
| promotion_last_5years   | Whether or not the employee was promoted in the last 5 years                                      |
| burnout                 | Flag: True if (number_project ≥ 6 or average_monthly_hours ≥ 240) and satisfaction_level ≤ 0.3    |

**Notes:**
- The "burnout" feature is a logical flag engineered to capture high-risk, overworked, and dissatisfied employees.

Best hyperparameters for **Random Forest (Core + Burnout)**:

```python
{
    'model__n_estimators': 300,          # Number of trees in the forest
    'model__min_samples_split': 2,       # Minimum samples to split an internal node
    'model__min_samples_leaf': 1,        # Minimum samples per leaf node
    'model__max_samples': 1.0,           # Use all samples for each tree
    'model__max_features': 1.0,          # Use all features for each split
    'model__max_depth': 5,               # Maximum depth of each tree
    'model__class_weight': 'balanced'    # Adjusts for class imbalance
}
```

---

In [41]:
confirm = pd.concat(
    [
        df_rf_winner,
        df_results[
            df_results["Model"].str.contains(
                "Random Forest (Core + Burnout)", regex=False
            )
        ],
    ]
)
confirm

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix,Best Params,CV Best Score,Search Time (s)
6,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.753266,0.939012,0.976981,7,"[[7508, 491], [94, 1499]]",NaN,NaN,NaN
6,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.753266,0.939012,0.976981,7,"[[7508, 491], [94, 1499]]","{'model__n_estimators': 300, 'model__min_sampl...",0.940991,78.417716


<a id="xgboost"></a>

## XGBoost

[Back to top](#)

<div class="emphasis-box">
  <strong>Model selection criteria aligned with both business and performance goals (in order):</strong>
  <ul>
    <li>A <strong>minimum recall threshold</strong> of 0.935</li>
    <li>A <strong>minimum F<sub>2</sub> score</strong> of 0.85 (prioritizing recall at a 4:1 weight over precision)</li>
    <li>Preference for models with <strong>fewer features</strong></li>
    <li>Tie-breaking by <strong>highest F<sub>2</sub></strong>, followed by <strong>precision</strong></li>
  </ul>
</div>

All XGBoost Models:

In [42]:
# display model evaluation results for all XGBoost models
df_xgb = df_results[df_results["Model"].str.contains("XGBoost")]
if df_xgb.empty:
    print("No XGBoost models found in results.")
# print("XGBoost Model Evaluation Results:")
df_xgb = df_xgb[
    [
        "Model",
        "Recall",
        "F2 Score",
        "F1 Score",
        "Precision",
        "Accuracy",
        "ROC AUC",
        "Num Features",
        "Confusion Matrix",
    ]
]
df_xgb

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
10,XGBoost (base),0.936598,0.926018,0.910589,0.885986,0.969454,0.986294,18,"[[7807, 192], [101, 1492]]"
11,XGBoost (Core + Burnout),0.936598,0.923953,0.905615,0.876616,0.967577,0.983741,7,"[[7789, 210], [101, 1492]]"
12,XGBoost with Binning,0.936598,0.924182,0.906165,0.877647,0.967786,0.985069,26,"[[7791, 208], [101, 1492]]"
13,XGBoost with Binning (feature selection),0.935970,0.926087,0.911648,0.888558,0.969871,0.984534,14,"[[7812, 187], [102, 1491]]"
14,XGBoost with Flags (feature selection),0.935342,0.926847,0.914391,0.894358,0.970913,0.983656,9,"[[7823, 176], [103, 1490]]"
17,XGBoost with Flags,0.934714,0.925420,0.911819,0.890018,0.969975,0.985902,21,"[[7815, 184], [104, 1489]]"
19,XGBoost with Interaction,0.934087,0.927565,0.917952,0.902365,0.972269,0.983251,22,"[[7838, 161], [105, 1488]]"
23,XGBoost (Core + Interactions + Burnout),0.932203,0.917346,0.895928,0.862369,0.964033,0.980458,6,"[[7762, 237], [108, 1485]]"
24,XGBoost with Interaction (feature selection),0.931576,0.925880,0.917465,0.903776,0.972164,0.983235,10,"[[7841, 158], [109, 1484]]"
26,XGBoost (Core + Interactions),0.930948,0.919747,0.903442,0.877515,0.966952,0.980297,6,"[[7792, 207], [110, 1483]]"


Above the minimum recall threshold of 0.935:

In [43]:
df_xgb_1 = df_xgb[df_xgb["Recall"] >= 0.935]
df_xgb_1

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
10,XGBoost (base),0.936598,0.926018,0.910589,0.885986,0.969454,0.986294,18,"[[7807, 192], [101, 1492]]"
11,XGBoost (Core + Burnout),0.936598,0.923953,0.905615,0.876616,0.967577,0.983741,7,"[[7789, 210], [101, 1492]]"
12,XGBoost with Binning,0.936598,0.924182,0.906165,0.877647,0.967786,0.985069,26,"[[7791, 208], [101, 1492]]"
13,XGBoost with Binning (feature selection),0.935970,0.926087,0.911648,0.888558,0.969871,0.984534,14,"[[7812, 187], [102, 1491]]"
14,XGBoost with Flags (feature selection),0.935342,0.926847,0.914391,0.894358,0.970913,0.983656,9,"[[7823, 176], [103, 1490]]"


Above the minimum F₂ threshold of 0.85:

In [44]:
df_xgb_2 = df_xgb_1[df_xgb_1["F2 Score"] >= 0.85]
df_xgb_2

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
10,XGBoost (base),0.936598,0.926018,0.910589,0.885986,0.969454,0.986294,18,"[[7807, 192], [101, 1492]]"
11,XGBoost (Core + Burnout),0.936598,0.923953,0.905615,0.876616,0.967577,0.983741,7,"[[7789, 210], [101, 1492]]"
12,XGBoost with Binning,0.936598,0.924182,0.906165,0.877647,0.967786,0.985069,26,"[[7791, 208], [101, 1492]]"
13,XGBoost with Binning (feature selection),0.935970,0.926087,0.911648,0.888558,0.969871,0.984534,14,"[[7812, 187], [102, 1491]]"
14,XGBoost with Flags (feature selection),0.935342,0.926847,0.914391,0.894358,0.970913,0.983656,9,"[[7823, 176], [103, 1490]]"


With the fewest features:

In [45]:
df_xgb_winner = df_xgb_2[df_xgb_2["Num Features"] == df_xgb_2["Num Features"].min()]
df_xgb_winner

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix
11,XGBoost (Core + Burnout),0.936598,0.923953,0.905615,0.876616,0.967577,0.983741,7,"[[7789, 210], [101, 1492]]"


And the winner is "XGBoost (Core + Burnout)"

In [46]:
''' 
Model, feature function, and parameter grid definitions for winning tree-based models
 (used to build pipelines) 
 located at beginning of Decision Tree section 
 '''
xgb_config = [
    cfg for cfg in rf_xgb_fe2_configs if cfg["name"] == "XGBoost (Core + Burnout)"
]
results_training = run_model_evaluation(
    xgb_config,
    results_df=results_training,
    scoring=scoring,
    search_type="random",
    n_iter=50,
)
results_training[results_training["model"] == "XGBoost (Core + Burnout)"]

Running model: XGBoost (Core + Burnout)...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Execution time for XGBoost (Core + Burnout): 16.06 seconds
Best parameters for XGBoost (Core + Burnout): {'model__subsample': 1.0, 'model__scale_pos_weight': 5.02134337727558, 'model__reg_lambda': 2, 'model__reg_alpha': 1, 'model__n_estimators': 100, 'model__min_child_weight': 5, 'model__max_depth': 3, 'model__learning_rate': 0.2, 'model__gamma': 0.1, 'model__colsample_bytree': 0.6}
Best score for XGBoost (Core + Burnout): 0.9366 (recall)
Model XGBoost (Core + Burnout) not saved. Set save_model=True to save it.



,model,recall,f2,f1,roc_auc,precision,accuracy,features,best_params,cv_best_score,conf_matrix,search_time
3,XGBoost (Core + Burnout),0.936598,0.923953,0.905615,0.983741,0.876616,0.967577,7,"{'model__subsample': 1.0, 'model__scale_pos_we...",0.936592,"[[7789, 210], [101, 1492]]",16.056963


#### Data Dictionary: XGBoost — "Core + Burnout" Features

This model uses a focused set of original features plus an engineered "burnout" flag.

| Variable                | Description                                                                                       |
|-------------------------|---------------------------------------------------------------------------------------------------|
| satisfaction_level      | Employee-reported job satisfaction level [0–1]                                                    |
| last_evaluation         | Score of employee's last performance review [0–1]                                                 |
| number_project          | Number of projects employee contributes to                                                        |
| average_monthly_hours   | Average number of hours employee worked per month                                                 |
| tenure                  | How long the employee has been with the company (years)                                           |
| promotion_last_5years   | Whether or not the employee was promoted in the last 5 years                                      |
| burnout                 | Flag: True if (number_project ≥ 6 or average_monthly_hours ≥ 240) and satisfaction_level ≤ 0.3    |

**Notes:**
- The "burnout" feature is a logical flag engineered to capture high-risk, overworked, and dissatisfied employees.

Best hyperparameters for **XGBoost (Core + Burnout)**:

```python
{
    'model__subsample': 1.0,                 # Use all rows for each tree
    'model__scale_pos_weight': 5.02134337727558, # Balances positive/negative classes
    'model__reg_lambda': 2,                  # L2 regularization term
    'model__reg_alpha': 1,                   # L1 regularization term
    'model__n_estimators': 100,              # Number of boosting rounds
    'model__min_child_weight': 5,            # Minimum sum of instance weight in a child
    'model__max_depth': 3,                   # Maximum tree depth
    'model__learning_rate': 0.2,             # Step size shrinkage
    'model__gamma': 0.1,                     # Minimum loss reduction to make a split
    'model__colsample_bytree': 0.6           # Fraction of features for each tree
}
```

---

In [47]:
confirm = pd.concat(
    [
        df_xgb_winner,
        df_results[
            df_results["Model"].str.contains("XGBoost (Core + Burnout)", regex=False)
        ],
    ]
)
confirm

,Model,Recall,F2 Score,F1 Score,Precision,Accuracy,ROC AUC,Num Features,Confusion Matrix,Best Params,CV Best Score,Search Time (s)
11,XGBoost (Core + Burnout),0.936598,0.923953,0.905615,0.876616,0.967577,0.983741,7,"[[7789, 210], [101, 1492]]",NaN,NaN,NaN
11,XGBoost (Core + Burnout),0.936598,0.923953,0.905615,0.876616,0.967577,0.983741,7,"[[7789, 210], [101, 1492]]","{'model__subsample': 1.0, 'model__scale_pos_we...",0.936592,16.96148


In [48]:
results_training.to_csv(
    "../results/winning_model_evaluation_results.csv", index=False
)
results_training

,model,recall,f2,f1,roc_auc,precision,accuracy,features,best_params,cv_best_score,conf_matrix,search_time
0,Logistic Regression with Binning (feature sele...,0.939535,0.855104,0.753531,0.947481,0.629004,0.896451,14,"{'model__C': 0.1, 'model__class_weight': 'bala...",0.939535,"[[6594, 834], [91, 1414]]",6.323299
1,Decision Tree (Core + Burnout),0.943503,0.887040,0.813972,0.956257,0.715714,0.928378,7,"{'model__class_weight': 'balanced', 'model__ma...",0.943495,"[[7402, 597], [90, 1503]]",3.033708
2,Random Forest (Core + Burnout),0.940992,0.896317,0.836729,0.976981,0.753266,0.939012,7,"{'model__n_estimators': 300, 'model__min_sampl...",0.940991,"[[7508, 491], [94, 1499]]",71.203241
3,XGBoost (Core + Burnout),0.936598,0.923953,0.905615,0.983741,0.876616,0.967577,7,"{'model__subsample': 1.0, 'model__scale_pos_we...",0.936592,"[[7789, 210], [101, 1492]]",16.056963


<a id="model-evaluation-results"></a>

## **Model Evaluation Results**

[Back to top](#)

## Recall evaluation metrics

- **AUC** is the area under the ROC curve; it's also considered the probability that the model ranks a random positive example more highly than a random negative example.
- **Precision** measures the proportion of data points predicted as True that are actually True, in other words, the proportion of positive predictions that are true positives.
- **Recall** measures the proportion of data points that are predicted as True, out of all the data points that are actually True. In other words, it measures the proportion of positives that are correctly classified.
- **Accuracy** measures the proportion of data points that are correctly classified.
- **F1-score** is an aggregation of precision and recall.






> ### Execute Stage Reflection
> 
> #### What key insights emerged from your model(s)?
> - **Satisfaction level** and **workload** (number of projects, monthly hours) are the strongest predictors of attrition.
> - Two main at-risk groups: **overworked/burned-out employees** (many projects, long hours, low satisfaction) and **underworked/disengaged employees** (few projects, low satisfaction).
> - **Tenure** is important: attrition peaks at 4–5 years, then drops sharply.
> - **Salary, department, and recent promotions** have minimal predictive value.
> - **Tree-based models (Random Forest, XGBoost)** achieved the best balance of recall, precision, and F1. With feature engineering, **logistic regression** became competitive and highly interpretable.
> 
> #### What business recommendations do you propose based on the models built?
> - **Monitor satisfaction and workload:** Regularly survey employees and track workload to identify those at risk of burnout or disengagement.
> - **Targeted retention efforts:** Focus on employees with low satisfaction and extreme workloads, especially those at the 4–5 year tenure mark.
> - **Promotions and recognition:** Consider more frequent recognition or advancement opportunities.
> - **Work-life balance:** Encourage reasonable project loads and monthly hours to reduce burnout risk.
> 
> #### What potential recommendations would you make to your manager/company?
> - **Implement early warning systems** using the model to flag at-risk employees for supportive HR outreach.
> - **Review workload distribution** and ensure fair, manageable assignments.
> - **Conduct stay interviews** with employees approaching 4–5 years of tenure.
> - **Communicate transparently** about how predictive models are used, emphasizing support rather than punitive action.
> 
> #### Do you think your model could be improved? Why or why not? How?
> - **Feature engineering:** Further refine interaction terms or add time-based features if available.
> - **External data:** Incorporate additional data (e.g., engagement surveys, manager ratings, exit interview themes).
> - **Model calibration:** Regularly retrain and calibrate the model as new data becomes available.
> - **Bias audits:** Routinely check for bias across demographic groups.
> 
> #### Given what you know about the data and the models you were using, what other questions could you address for the team?
> - What are the specific reasons for attrition in different departments or roles?
> - Are there seasonal or cyclical patterns in attrition?
> - How do external factors (e.g., economic conditions, industry trends) affect turnover?
> - What interventions are most effective for retaining at-risk employees?
> 
> #### What resources do you find yourself using as you complete this stage? (Make sure to include the links.)
> - [pandas documentation](https://pandas.pydata.org/docs/)
> - [matplotlib documentation](https://matplotlib.org/stable/users/index.html)
> - [seaborn documentation](https://seaborn.pydata.org/)
> - [scikit-learn documentation](https://scikit-learn.org/stable/user_guide.html)
> - [XGBoost documentation](https://xgboost.readthedocs.io/en/stable/)
> - [Kaggle HR Analytics Dataset](https://www.kaggle.com/datasets/mfaisalqureshi/hr-analytics-and-job-prediction?select=HR_comma_sep.csv)
> 
> #### Do you have any ethical considerations in this stage?
> - **Data privacy:** Ensure employee data is kept confidential and secure.
> - **Fairness:** Avoid using the model to unfairly target or penalize specific groups.
> - **Transparency:** Clearly communicate how predictions are generated and used.
> - **Supportive use:** Use predictions to offer support and resources, not for punitive measures.
> - **Ongoing monitoring:** Regularly audit the model for bias and unintended consequences.

### Conclusion, Recommendations, Next Steps

> ### Conclusion
> - **Satisfaction level** and **workload** (number of projects, monthly hours) are the strongest predictors of employee attrition.
> - Two main at-risk groups emerged: **overworked/burned-out employees** (many projects, long hours, low satisfaction) and **underworked/disengaged employees** (few projects, low satisfaction).
> - **Tenure** is important: attrition peaks at 4–5 years, then drops sharply.
> - **Salary, department, and recent promotions** have minimal predictive value.
> - **Tree-based models (Random Forest, XGBoost)** achieved the best balance of recall, precision, and F1. With feature engineering, **logistic regression** became competitive and highly interpretable.
> 
> ### Recommendations
> - **Monitor satisfaction and workload:** Regularly survey employees and track workload to identify those at risk of burnout or disengagement.
> - **Targeted retention efforts:** Focus on employees with low satisfaction and extreme workloads, especially those at the 4–5 year tenure mark.
> - **Promotions and recognition:** Consider more frequent recognition or advancement opportunities.
> - **Work-life balance:** Encourage reasonable project loads and monthly hours to reduce burnout risk.
> - **Implement early warning systems:** Use the model to flag at-risk employees for supportive HR outreach.
> - **Review workload distribution:** Ensure fair, manageable assignments.
> - **Conduct stay interviews:** Engage employees approaching 4–5 years of tenure.
> - **Communicate transparently:** Clearly explain how predictive models are used, emphasizing support rather than punitive action.
> 
> ### Next Steps
> - **Model deployment:** Integrate the predictive model into HR processes for early identification of at-risk employees.
> - **Continuous improvement:** Regularly retrain and calibrate the model as new data becomes available.
> - **Expand data sources:** Incorporate additional data (e.g., engagement surveys, manager ratings, exit interview themes) to improve model accuracy.
> - **Bias and fairness audits:** Routinely check for bias across demographic groups and monitor for unintended consequences.
> - **Ethical safeguards:** Ensure employee data privacy, fairness, and transparency in all predictive analytics initiatives.
> 
> ---
> **Resources Used:**
> - [pandas documentation](https://pandas.pydata.org/docs/)
> - [matplotlib documentation](https://matplotlib.org/stable/users/index.html)
> - [seaborn documentation](https://seaborn.pydata.org/)
> - [scikit-learn documentation](https://scikit-learn.org/stable/user_guide.html)
> - [XGBoost documentation](https://xgboost.readthedocs.io/en/stable/)
> - [Kaggle HR Analytics Dataset](https://www.kaggle.com/datasets/mfaisalqureshi/hr-analytics-and-job-prediction?select=HR_comma_sep.csv)

<a id="appendix-data-dictionary"></a>

# Appendix: Data Dictionary
---

[Back to top](#)

The dataset contains 15,000 rows and 10 columns for the variables listed below. 

**Note:** For more information about the data, refer to its source on [Kaggle](https://www.kaggle.com/datasets/mfaisalqureshi/hr-analytics-and-job-prediction?select=HR_comma_sep.csv).

Variable  |Description |
-----|-----|
satisfaction_level|Employee-reported job satisfaction level [0&ndash;1]|
last_evaluation|Score of employee's last performance review [0&ndash;1]|
number_project|Number of projects employee contributes to|
average_monthly_hours|Average number of hours employee worked per month|
time_spend_company|How long the employee has been with the company (years)
Work_accident|Whether or not the employee experienced an accident while at work
left|Whether or not the employee left the company
promotion_last_5years|Whether or not the employee was promoted in the last 5 years
Department|The employee's department
salary|The employee's salary (U.S. dollars)

### Feature Engineering Data Dictionary


The following table describes the engineered features created for model development. These features are derived from the original dataset using binning, interaction terms, and logical flags to capture important patterns identified during exploratory data analysis.

Variable  |Description |
-----|-----|
**Bins** | |
satisfaction_bin_low | Binary indicator: satisfaction_level is low (≤ 0.4) |
satisfaction_bin_medium | Binary indicator: satisfaction_level is medium (> 0.4 and ≤ 0.7) |
satisfaction_bin_high | Binary indicator: satisfaction_level is high (> 0.7) |
hours_bin_low | Binary indicator: average_monthly_hours is low (≤ 160) |
hours_bin_medium | Binary indicator: average_monthly_hours is medium (> 160 and ≤ 240) |
hours_bin_high | Binary indicator: average_monthly_hours is high (> 240) |
projects_bin_low | Binary indicator: number_project is low (≤ 2) |
projects_bin_medium | Binary indicator: number_project is medium (> 2 and ≤ 5) |
projects_bin_high | Binary indicator: number_project is high (> 5) |
tenure_bin_short | Binary indicator: tenure is short (≤ 3 years) |
tenure_bin_mid | Binary indicator: tenure is mid (> 3 and ≤ 5 years) |
tenure_bin_long | Binary indicator: tenure is long (> 5 years) |
**Interactions** | |
satisfaction_x_projects | Interaction: satisfaction_level × number_project |
satisfaction_x_hours | Interaction: satisfaction_level × average_monthly_hours |
evaluation_x_satisfaction | Interaction: last_evaluation × satisfaction_level |
hours_per_project | Ratio: average_monthly_hours divided by number_project |
**Flags** | |
burnout | Flag: True if (number_project ≥ 6 or average_monthly_hours ≥ 240) and satisfaction_level ≤ 0.3 |
disengaged | Flag: True if (number_project ≤ 2 and average_monthly_hours < 160 and satisfaction_level ≤ 0.5) |
no_promo_4yr | Flag: True if promotion_last_5years == 0 and tenure ≥ 4 |

**Note:**  
Binned features are one-hot encoded as separate columns (e.g., satisfaction_bin_low, satisfaction_bin_medium, satisfaction_bin_high). Only the relevant dummy variables (excluding the first category for each bin) are included in the final dataset, depending on the encoding strategy.

<a href id="appendix-feature-engineering"></a>

# Feature Engineering
---

[Back to Top](#)

In addition to baseline models run with all original features, all feature engineering varieties below were tested with all four model types: **Logistic Regression**, **Decision Tree**, **Random Forest**, and **XGBoost**.

### Feature Engineering Round One

**Original features + binning**:
This set adds binned versions of key variables (satisfaction, workload, projects, tenure) to the original features. Binning helps capture non-linear effects and threshold-based risk groups identified in EDA (e.g., low/medium/high satisfaction, extreme workloads, tenure windows).

**Original features + interactions**:
This set augments the original features with interaction terms that capture relationships between satisfaction, workload, and performance (e.g., satisfaction × projects, satisfaction × hours, evaluation × satisfaction, hours per project). These interactions help distinguish between disengaged, overworked, and healthy employees.

**Original features + categorical flags**:
This set adds logical flag features to the original data, such as "burnout" (overworked and dissatisfied), "disengaged" (underworked and dissatisfied), and "no_promo_4yr" (long tenure without promotion). These flags directly encode high-risk employee profiles identified in EDA.

**Feature selection**:
For each of the above, a feature selection variant was also run, dropping weak predictors (department, salary, work_accident) to reduce noise and multicollinearity, especially for logistic regression.

### Feature Engineering Round Two

**Selected features + burnout flag:**
This set isolates the core predictors of attrition (satisfaction, workload, tenure, promotion) and adds a “burnout” flag to capture the high-risk group of overworked, dissatisfied employees

**Selected features + interactions:**
This set focuses on the main drivers (satisfaction, workload, tenure) and adds interaction terms (satisfaction × projects, hours per project) to capture non-linear effects and workload intensity, which EDA showed are important for distinguishing between underworked, overworked, and healthy employees.

**Selected features + interactions + burnout flag:**
This feature set combines the core predictors of attrition (satisfaction, workload, tenure) with a “burnout” flag to capture high-risk, overworked employees. It also includes a key interaction term, "satisfaction × projects", to distinguish between groups identified in EDA

<a href id="appendix-hyperparameters"></a>

## Appendix: Hyperparameter Tuning Grids
---

[Back to Top](#)

This section summarizes the hyperparameter grids used for model selection and tuning via `GridSearchCV` and `RandomizedSearchCV` for all baseline and feature-engineered models.

---

### **Logistic Regression**

**Baseline Grid:**
```python
lr_base_param_grid = {
    "Logistic Regression": {
        "model__C": [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization strength (inverse)
        "model__penalty": ["l1", "l2"],             # L1 (Lasso) or L2 (Ridge)
        "model__solver": ["liblinear"],             # Supports L1/L2
        "model__class_weight": [None, "balanced"],  # Handle class imbalance
    }
}
```

**Feature Engineering Grid:**
```python
lr_fe_params = {
    "model__C": [0.1, 1.0, 10.0],
    "model__penalty": ["l1", "l2"],
    "model__solver": ["liblinear"],
    "model__class_weight": [None, "balanced"],
}
```

---

### **Decision Tree**

**Baseline Grid:**
```python
tree_base_param_grids = {
    "Decision Tree": {
        "model__max_depth": [4, 6, 8, None],        # Max tree depth
        "model__min_samples_leaf": [1, 2, 5],       # Min samples at leaf
        "model__min_samples_split": [2, 4, 6],      # Min samples to split
        "model__class_weight": [None, "balanced"],  # Handle class imbalance
    },
    ...
}
```

**Feature Engineering Grid:**
```python
tree_fe_params = {
    "Decision Tree": {
        "model__max_depth": [3, 4, 5, 6, 8],
        "model__min_samples_leaf": [1, 2, 3],
        "model__min_samples_split": [2, 3, 4],
        "model__class_weight": [None, "balanced"],
    },
    ...
}
```

---

### **Random Forest**

**Baseline Grid:**
```python
tree_base_param_grids = {
    "Random Forest": {
        "model__n_estimators": [300, 500],          # Number of trees
        "model__max_depth": [3, 5, None],           # Max tree depth
        "model__max_features": ["sqrt", 1.0],       # Features considered at each split
        "model__max_samples": [0.7, 1.0],           # Fraction of samples per tree
        "model__min_samples_leaf": [1, 2, 3],       # Min samples at leaf
        "model__min_samples_split": [2, 3, 4],      # Min samples to split
        "model__class_weight": [None, "balanced"],  # Handle class imbalance
    },
    ...
}
```

**Feature Engineering Grid:**
```python
tree_fe_params = {
    "Random Forest": {
        "model__n_estimators": [100, 300],
        "model__max_depth": [3, 4, 5, 8],
        "model__max_features": ["sqrt", 1.0],
        "model__max_samples": [0.7, 1.0],
        "model__min_samples_leaf": [1, 2],
        "model__min_samples_split": [2, 3],
        "model__class_weight": [None, "balanced"],
    },
    ...
}
```

---

### **XGBoost**

**Baseline Grid:**
```python
tree_base_param_grids = {
    "XGBoost": {
        "model__n_estimators": [100, 300],              # Number of boosting rounds
        "model__max_depth": [3, 5, 7],                  # Max tree depth
        "model__learning_rate": [0.01, 0.1, 0.2],       # Step size shrinkage
        "model__subsample": [0.6, 0.8, 1.0],            # Row subsampling
        "model__colsample_bytree": [0.6, 0.8, 1.0],     # Feature subsampling
        "model__min_child_weight": [1, 5, 10],          # Min sum of instance weight in a child
        "model__gamma": [0, 0.1, 0.2],                  # Min loss reduction to split
        "model__scale_pos_weight": [1, scale_pos_weight_value],  # Class imbalance
    },
    ...
}
```

**Feature Engineering Grid:**
```python
tree_fe_params = {
    "XGBoost": {
        "model__n_estimators": [100, 300],
        "model__max_depth": [3, 4, 5, 8],
        "model__learning_rate": [0.1, 0.2],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__colsample_bytree": [0.6, 0.8, 1.0],
        "model__min_child_weight": [1, 5],
        "model__gamma": [0, 0.1, 0.2],
        "model__scale_pos_weight": [1, scale_pos_weight_value],
        "model__reg_alpha": [0, 0.1, 1],            # L1 regularization
        "model__reg_lambda": [1, 2, 5],             # L2 regularization
    },
    ...
}
```

---

**Notes:**
- All hyperparameters were tuned using either `GridSearchCV` (for smaller grids or faster models) or `RandomizedSearchCV` (for larger grids or slower models).
- `scale_pos_weight_value` is dynamically calculated as the ratio of negative to positive samples in the training set to address class imbalance for XGBoost.
- For all models, `model__class_weight="balanced"` was included as an option to help mitigate class imbalance.
- Feature engineering grids used slightly smaller or more regularized search spaces for efficiency.

---



<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.bundle.min.js"></script>
<script src="../static/js/scripts.js"></script>

